# Regularization techniques

In [150]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# read csv and save output as numpy array
data = pd.read_csv('insurance.csv', delimiter=';')
data.head() # print top 5 rows

,age,sex,bmi,children,smoker,region,shoesize,charges
0,19,female,27.900,0,yes,southwest,36,16884.92400
1,18,male,33.770,1,no,southeast,43,1725.55230
2,28,male,33.000,3,no,southeast,43,4449.46200
3,33,male,22.705,0,no,northwest,41,21984.47061
4,32,male,28.880,0,no,northwest,42,3866.85520


In [3]:
# Starting by applying feature standartization using built-in sklearn function on numerical columns (except columns "charges" which will be a target column)

# https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/

num_cols = ['age', 'bmi', 'children', 'shoesize']

data_scaled = data.copy()

for i in num_cols:
    # define scaling method
    scale = StandardScaler().fit(data[[i]])
    # transform data columns
    data_scaled[i] = scale.transform(data[[i]])

In [4]:
# Converting categorical variables into numerical arrays
sexes = pd.get_dummies(data_scaled.sex)
smokers = pd.get_dummies(data_scaled.smoker, prefix='smoker')
regions = pd.get_dummies(data_scaled.region, prefix='region')

In [5]:
# Concatenate arrays with existing dataframe, drop original categorical variables
df = data_scaled.join([sexes, smokers, regions]).drop(columns=['sex', 'smoker', 'region'])

In [6]:
# Check dataframe variable types
df.dtypes

age                 float64
bmi                 float64
children            float64
shoesize            float64
charges             float64
female                uint8
male                  uint8
smoker_no             uint8
smoker_yes            uint8
region_northeast      uint8
region_northwest      uint8
region_southeast      uint8
region_southwest      uint8
dtype: object

In [7]:
df.describe()

,age,bmi,children,shoesize,charges,female,male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
count,1.338000e+03,1.338000e+03,1.338000e+03,1.338000e+03,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,-1.805565e-16,-2.124194e-16,-5.576008e-17,2.283508e-16,13270.422265,0.494768,0.505232,0.795217,0.204783,0.242152,0.242900,0.272048,0.242900
std,1.000374e+00,1.000374e+00,1.000374e+00,1.000374e+00,12110.011237,0.500160,0.500160,0.403694,0.403694,0.428546,0.428995,0.445181,0.428995
min,-1.509965e+00,-2.412011e+00,-9.086137e-01,-1.364052e+00,1121.873900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-8.691547e-01,-7.164063e-01,-9.086137e-01,-7.568991e-01,4740.287150,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-1.474046e-02,-4.320880e-02,-7.876719e-02,1.538302e-01,9382.033000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.396738e-01,6.611572e-01,7.510793e-01,7.609831e-01,16639.912515,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.765289e+00,3.685522e+00,3.240619e+00,1.671713e+00,63770.428010,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# Split feature and target from the training dataset
X = df.drop(columns = 'charges')
y = df.charges

In [152]:
# Define function for model cross-validation and model evaluation

def func(function, X, y):
    rmse = cross_val_score(function, X, y, cv=5, scoring='neg_root_mean_squared_error')
    rmse = -np.round(np.mean(rmse))
    score = cross_val_score(function, X, y, cv=5, scoring='r2')
    score = np.round(np.mean(score), 3)
    print(f'RMSE = {rmse} | R2 score = {score}')

In [105]:
# Define and evaluate baseline model
linreg = LinearRegression()
print('Standard linear regression model')
func(linreg, X, y)

Standard linear regression model
RMSE = 6070.0 | R2 score = 0.747


In [106]:
# Define and evaluate Lasso regression model 
lasso = Lasso()
print('Lasso linear regression model')
func(lasso, X, y)

Lasso linear regression model
RMSE = 6070.0 | R2 score = 0.747


In [107]:
# Define and evaluate Ridge regression model
ridge = Ridge()
print('Ridge linear regression model')
func(ridge, X, y)

Ridge linear regression model
RMSE = 6070.0 | R2 score = 0.747


We do clearly receive the same results as no further regularization is required to the model of polynominal degree 1

In [110]:
# Add polinominal features 
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

#Define loop to generate polynominal features with degree 1,2 and 3 for linear regression model:
for i in range(1, 4):
    poly = PolynomialFeatures(i)
    X_transformed =  poly.fit_transform(X)
    linreg = LinearRegression()
    print(f'Standard linear regression model polinominal degree {i}')
    func(linreg, X_transformed, y)

Standard linear regression model polinominal degree 1
RMSE = 6070.0 | R2 score = 0.747
Standard linear regression model polinominal degree 2
RMSE = 4934.0 | R2 score = 0.832
Standard linear regression model polinominal degree 3
RMSE = 5138.0 | R2 score = 0.818


We clearly see some model improvement using polynominal degree 2

In [254]:
# define function to iterate through polynominal degree and provided alpha parameter
# return best combination for polynominal degree and alpha parameter
# https://scikit-learn.org/stable/auto_examples/linear_model/plot_ridge_path.html

def poly_func(function):
    n_alphas = 5
    alphas = np.logspace(-4, 0, n_alphas) # define range for alpha iterations
    coefs = np.empty([1,4])
    for i in range(1, 4):
        for j in alphas:
            poly = PolynomialFeatures(i)
            X_transformed =  poly.fit_transform(X) # creat en extended matrix with additional polynominal features
            func = function(max_iter=1000000, alpha= j) # define function
            # calculate rmse and r2 scores
            rmse = cross_val_score(func, X_transformed, y, cv=5, scoring='neg_root_mean_squared_error')
            rmse = -np.round(np.mean(rmse))
            score = cross_val_score(func, X_transformed, y, cv=5, scoring='r2')
            score = np.round(np.mean(score), 3)
            # add scores to array
            coefs =  np.vstack((coefs, [i, j, rmse, score]))
    # find row index with lowest value of rmse        
    index = np.where(coef[:,2] == np.min(coef[:,2]))
    # return row
    row = coef[index]
    print (f'lowest rmse = {row[0,2]} and r2 score = {row[0,3]} is achivied using polynominal degree {row[0,0]} and alpha = {row[0,1]}')

In [255]:
# Apply function to Ridge linear model
poly_func(Ridge)

lowest rmse = 4913.0 and r2 score = 0.833 is achivied using plonominal degree 2.0 and alpha = 1.0


We see some improvement comparing to a regular linear model at the polynominal degree 2 using Ridge model. Further model optimization will be needed to achive better model evalulation results.